**This notebook generates videos of the uncertainties predicted by PixLoc on sequences of images.**

In [1]:
import torch
from torch.nn.functional import interpolate
import numpy as np
from tqdm import tqdm
from collections import defaultdict
from datetime import datetime
import matplotlib.pyplot as plt

from pixloc.utils.data import Paths
from pixloc.localization.feature_extractor import FeatureExtractor
from pixloc.pixlib.utils.experiments import load_experiment
from pixloc.pixlib.datasets.view import read_image
from pixloc.visualization.viz_2d import plot_images, add_text
from pixloc.visualization.animation import VideoWriter, display_video
from pixloc.settings import DATA_PATH, LOC_PATH

# Datasets
Run one of the following cells to parse the list of images in a sequence.

### 1) CMU

In [2]:
from pixloc.run_CMU import default_paths, experiment

paths = default_paths.add_prefixes(DATA_PATH/'CMU', LOC_PATH/'CMU')
date2query = defaultdict(list)
for slice_ in range(2, 26):
    queries = sorted(paths.interpolate(slice=slice_).query_images.glob('*.jpg'))
    for q in queries:
        ts = int(q.name.split('_')[-1].rstrip('us.jpg'))
        date = datetime.utcfromtimestamp(ts//1000000)
        date2query[date.strftime('%Y-%m-%d')].append((ts, q))
date2query = dict(date2query)

print('Sequences:', sorted(date2query.keys()))
seq = '2010-12-21'
cam = 'c1'
images = [p for _, p in date2query[seq] if cam in p.name]
# plt.plot([ts for ts, p in date2query[seq] if cam in p.name]);

Sequences: ['2010-09-01', '2010-09-15', '2010-10-01', '2010-10-19', '2010-10-26', '2010-11-03', '2010-11-12', '2010-11-22', '2010-12-21', '2011-03-04', '2011-07-28']


### 2) Cambridge

In [150]:
from pixloc.run_Cambridge import default_paths, experiment

paths = default_paths.add_prefixes(DATA_PATH/'Cambridge', LOC_PATH/'Cambridge')
paths = paths.interpolate(scene='OldHospital')
all_sequences = {}
for seq in paths.query_images.glob('seq*'):
    all_sequences[f'{seq.name}'] = sorted(seq.iterdir())
    
print('Sequences:', {s: len(l) for s, l in all_sequences.items()})
cam = 'seq1'
images = all_sequences[cam]

Sequences: {'seq3': 87, 'seq2': 161, 'seq1': 301, 'seq9': 122, 'seq5': 68, 'seq6': 87, 'seq8': 126, 'seq4': 56, 'seq7': 76}


### 3) Aachen v1.1

In [152]:
from pixloc.run_Aachen import default_paths, experiment

paths = default_paths.add_prefixes(DATA_PATH/'Aachen_v1.1', LOC_PATH/'Aachen_v1.1')
root = paths.reference_images / 'sequences'
all_sequences = {}
all_sequences['gopro'] = sorted((root/'gopro3_undistorted').glob('*.png'))
for seq in (root/'nexus4_sequences').iterdir():
    all_sequences[f'nexus_{seq.name.split("_")[1]}'] = sorted(seq.iterdir())

print('Sequences:', {s: len(l) for s, l in all_sequences.items()})
cam = 'gopro'
images = all_sequences[cam]

Sequences: {'gopro': 1046, 'nexus_1': 171, 'nexus_3': 17, 'nexus_7': 242, 'nexus_6': 84, 'nexus_2': 43, 'nexus_5': 203, 'nexus_8': 71, 'nexus_4': 494}


### 4) RobotCar

In [118]:
from pixloc.run_RobotCar import default_paths, experiment

paths = default_paths.add_prefixes(DATA_PATH/'RobotCar', LOC_PATH/'RobotCar')
condition = 'snow'
cam = 'right'
images = sorted((paths.query_images/condition/cam).glob('*.jpg'))

# Inference

Choose one of the two models pretrained on CMU or MegaDepth:

In [3]:
device = torch.device('cuda')
pipeline = load_experiment(experiment).to(device).eval()
net = FeatureExtractor(pipeline.extractor, device, {'resize': 1024})

[11/23/2021 08:36:51 pixloc.pixlib.utils.experiments INFO] Loading checkpoint checkpoint_best.tar


- Set `preview=True` to visualize the predictions for a few images in the sequence.
- Set `preview=False` to dump a video of the predictions.

In [7]:
preview = False

video = 'uncertainties.mp4'
args = dict(cmaps='turbo', dpi=50)
if preview:
    num = slice(10)
    skip = slice(None, None, 100)
else:
    num = slice(100)
    skip = slice(None, None, 10)
    writer = VideoWriter('./tmp')

for i, path in enumerate(tqdm(images[skip][num])):
    image = read_image(path)
    _, _, confs = net(image)
    shape = confs[0][0].shape
    confs = [interpolate(c[None], size=shape, mode='bilinear', align_corners=False)[0] for c in confs]
    confs = [c.cpu().numpy()[0] for c in confs]
    fine, mid, coarse = confs
    
    # fuse the confidence maps (for visualization only)
    fused = (fine*mid*coarse)**(1/3)

    if preview:
        plot_images([image, fine, mid, coarse, fused], **args)
    else:
        plot_images([image, fused], **args)
    
    # add image as background
    [a.images[0].set(alpha=0.8) for a in plt.gcf().axes[1:]];
    [a.imshow(image, extent=a.images[0]._extent, zorder=-1) for a in plt.gcf().axes[1:]];
    
    if not preview:
        writer.add_frame()

if not preview:
    writer.to_video(video, fps=3, crf=23)

100%|██████████████████████████████████████████████████████████████| 100/100 [00:33<00:00,  3.02it/s]
[11/23/2021 08:38:53 pixloc.visualization.animation INFO] Running ffmpeg.


FileNotFoundError: [Errno 2] No such file or directory: 'ffmpeg'

In [ ]:
display_video(video)